## Librerias y conexión a modelo existente

In [2]:
import pandas as pd
import numpy as np
import comtypes.client
import sys
import matplotlib.pyplot as plt
import os
import math

def connect_to_existing_etabs():
    
        helper = comtypes.client.CreateObject('ETABSv1.Helper');
        helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper);

        try:
            myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject");
            print("Conectado exitosamente")
        except (OSError,comtypes.COMError):
            print("Ha fallado.")
            sys.exit(-1);

        SapModel = myETABSObject.SapModel
        return SapModel,myETABSObject,helper;

SapModel, myETABSObject, helper = connect_to_existing_etabs()

Conectado exitosamente


## Guardar y correr

In [2]:
# Guardar el modelo en la carpeta especificada
# Ese número crea una carpeta rara, no utilizar números
save_path = "C:\CSi_ETABS_API_Example"
model_file_name = "MOD3.edb"
model_file_path = os.path.join(save_path, model_file_name)
ret = SapModel.File.Save(model_file_path)

print(ret)

#run model (this will create the analysis model)
ret = SapModel.Analyze.RunAnalysis()

0


### Resultados por código Results: Cortante basal, deriva máxima y fuerza interna en elementos frame

#### CORTANTE BASAL

In [7]:
NumberResults = 0
LoadCase =[]
StepType =[]
StepNum =[]
Fx =[]
Fy =[]
Fz =[]
Mx =[]
ParamMy =[]
Mz =[]
gx = 0
gy = 0
gz = 0



#deselect all cases and combos
ret = SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()

#set case selected for output
ret = SapModel.Results.Setup.SetCaseSelectedForOutput("DEAD")

#get base reactions
ret = SapModel.Results.BaseReact(NumberResults, LoadCase, StepType, StepNum, Fx, Fy, Fz, Mx, ParamMy, Mz, gx, gy, gz)

ret

[1,
 ('Dead',),
 ('Single Value',),
 (0.0,),
 (1.6768808563938364e-11,),
 (3.097966327914037e-12,),
 (1497635.8201035487,),
 (294810200.8077847,),
 (-309550710.8481743,),
 (-4.802132025361061e-09,),
 0.0,
 0.0,
 0.0,
 0]

#### DERIVA MÁXIMA

In [5]:
NumberResults = 0
Story = []
LoadCase = []
StepType = []
StepNum = []
Direction = []
Drift = []
Label = []
X = []
Y = []
Z = []


#deselect all cases and combos
ret = SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()

#set case selected for output
ret = SapModel.Results.Setup.SetCaseSelectedForOutput("DEAD")

#get base reactions
ret = SapModel.Results.StoryDrifts(NumberResults, Story, LoadCase, StepType, StepNum, Direction, Drift, Label, X, Y, Z)

#get base reactions
[NumberResults, Story, LoadCase, StepType, StepNum, Direction, Drift, Label, X, Y, Z,ret] = SapModel.Results.StoryDrifts(NumberResults, Story, LoadCase, StepType, StepNum, Direction, Drift, Label, X, Y, Z)

ret
Story

('Story5', 'Story4', 'Story3', 'Story2', 'Story1', 'Story1')

#### FUERZA INTERNA EN FRAME

In [6]:
NumberResults = 0
Obj = []
ObjSta = []
Elm = []
ElmSta = []
LoadCase = []
StepType = []
StepNum = []
P = []
V2 = []
V3 = []
T = []
M2 = []
M3 = []


#deselect all cases and combos
ret = SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()

#set case selected for output
ret = SapModel.Results.Setup.SetCaseSelectedForOutput("DEAD")

#get frame forces
ret = SapModel.Results.FrameForce("176", 0, NumberResults, Obj, ObjSta, Elm, ElmSta, LoadCase, StepType, StepNum, P, V2, V3, T, M2, M3)

ret

[3,
 ('176', '176', '176'),
 (0.0, 1.6952000640000011, 3.3904001280000022),
 ('176', '176', '176'),
 (0.0, 1.6952000640000011, 3.3904001280000022),
 ('Dead', 'Dead', 'Dead'),
 ('Single Value', 'Single Value', 'Single Value'),
 (0.0, 0.0, 0.0),
 (-25.05558511604894, -24.204161790514547, -23.352738464980156),
 (-0.1547606103459369, -0.1547606103459369, -0.1547606103459369),
 (0.0995203322845565, 0.0995203322845565, 0.0995203322845565),
 (5.3109858603488546e-17, 5.3109858603488546e-17, 5.3109858603488546e-17),
 (0.12894233357834434, -0.03976454007973722, -0.20847141373781877),
 (-0.20051374212614811, 0.06183645443696334, 0.32418665100007477),
 0]

## Resultados obtenidos desde la Tabla de Resultados

In [17]:
# Ton_m_C = 12
tonf_m_C = 12
ret = SapModel.SetPresentUnits(tonf_m_C)

name_table="Base Reactions"
#name_table = "Modal Participating Mass Ratios"
table = SapModel.DatabaseTables.GetTableForDisplayArray(name_table ,GroupName="")    
cols=table[2]
noOfRows=table[3]
vals=np.array_split(table[4],noOfRows)
df= pd.DataFrame(vals)
df.columns=cols
df

,OutputCase,CaseType,StepType,StepNumber,StepLabel,FX,FY,FZ,MX,MY,MZ,X,Y,Z
0,Dead,LinStatic,None,None,None,0,0,679.3162,3396.5811,-3566.4102,0,0,0,0


In [23]:
name_table = "Modal Participating Mass Ratios"
table = SapModel.DatabaseTables.GetTableForDisplayArray(name_table ,GroupName="")    
cols=table[2]
noOfRows=table[3]
vals=np.array_split(table[4],noOfRows)
df= pd.DataFrame(vals)
df.columns=cols
df

,Case,Mode,Period,UX,UY,UZ,SumUX,SumUY,SumUZ,RX,RY,RZ,SumRX,SumRY,SumRZ
0,Modal,1,0.382,0.9212,0,0,0.9212,0,0,0,0.1166,0,0,0.1166,0
1,Modal,2,0.368,0,0.924,0,0.9212,0.924,0,0.1138,0,0,0.1138,0.1166,0
2,Modal,3,0.322,0,0,0,0.9212,0.924,0,0,0,0.9315,0.1138,0.1166,0.9315
3,Modal,4,0.123,0.0664,0,0,0.9877,0.924,0,0,0.8337,0,0.1138,0.9503,0.9315
4,Modal,5,0.118,0,0.0655,0,0.9877,0.9895,0,0.8413,0,0,0.9551,0.9503,0.9315
5,Modal,6,0.103,0,0,0,0.9877,0.9895,0,0,0,0.0575,0.9551,0.9503,0.989
6,Modal,7,0.069,0.0098,0,0,0.9975,0.9895,0,0,0.0262,0,0.9551,0.9765,0.989
7,Modal,8,0.066,0,0.0085,0,0.9975,0.998,0,0.0243,0,0,0.9794,0.9765,0.989
8,Modal,9,0.059,0,0,0,0.9975,0.998,0,0,0,0.0088,0.9794,0.9765,0.9979
9,Modal,10,0.048,0.0021,0,0,0.9996,0.998,0,0,0.0224,0,0.9794,0.9989,0.9979


In [25]:
df = df.drop(df.columns[[5,6,7,8, 9,10,12,13,14]], axis=1)
df

,Case,Mode,Period,UX,UY,RZ
0,Modal,1,0.382,0.9212,0,0
1,Modal,2,0.368,0,0.924,0
2,Modal,3,0.322,0,0,0.9315
3,Modal,4,0.123,0.0664,0,0
4,Modal,5,0.118,0,0.0655,0
5,Modal,6,0.103,0,0,0.0575
6,Modal,7,0.069,0.0098,0,0
7,Modal,8,0.066,0,0.0085,0
8,Modal,9,0.059,0,0,0.0088
9,Modal,10,0.048,0.0021,0,0


### Resultados de la tabla obtenidos por paso

In [ ]:
ret=SapModel.DatabaseTables.SetOutputOptionsForDisplay(False,0,0,0,True,0,0,False,0,0,2,2,2,2,1)
ret

## Interactive Database

In [29]:
NumberTables=-1
TableKey=[]
TableName=[]
ImportType=[]
ReturnValue=-1

[NumberTables,Tablekey, Tablename, ImportType,ret]=SapModel.DatabaseTables.GetAvailableTables(NumberTables,TableKey,TableName,ImportType)
ret
#NumberTables
Tablename
#Tablekey
#ImportType

('Analysis Options - Active Degrees of Freedom',
 'Analysis Options - Analysis Model for Nonlinear Hinges',
 'Analysis Options - Automatic Mesh Settings for Floors',
 'Analysis Options - Automatic Rectangular Mesh Options for Walls',
 'Analysis Options - Cracking Analysis Options',
 'Analysis Options - Design and Response Recovery Options',
 'Analysis Options - SAPFire Options',
 'Area Assignments - Auto Edge Constraints',
 'Area Assignments - Floor Auto Mesh Options',
 'Area Assignments - Insertion Point',
 'Area Assignments - Section Properties',
 'Area Assignments - Summary',
 'Area Section Property Definitions - Summary',
 'Beam Bays',
 'Beam Object Connectivity',
 'Column Bays',
 'Column Object Connectivity',
 'Composite Beam Design Preferences - AISC 360-16',
 'Composite Column Design Preferences - AISC 360-16',
 'Concrete Beam Overwrites - ACI 318-19',
 'Concrete Column Overwrites - ACI 318-19',
 'Concrete Frame Design Preferences - ACI 318-19',
 'Concrete Slab Design Overwrites

In [30]:
table_key='Load Pattern Definitions - Auto Seismic - User Coefficient'

table_data=SapModel.DatabaseTables.GetTableForEditingArray(table_key,'')

table_data

[1,
 ('Name',
  'IsAuto',
  'XDir',
  'XDirPlusE',
  'XDirMinusE',
  'YDir',
  'YDirPlusE',
  'YDirMinusE',
  'EccRatio',
  'TopStory',
  'BotStory',
  'OverStory',
  'OverDiaph',
  'OverEcc',
  'C',
  'K'),
 1,
 ('EQ',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  '0.05',
  'Story5',
  'Base',
  '',
  '',
  '',
  '0.1',
  '1'),
 0]

In [3]:
TableVersion = 1
NumberRecords = 1
tTableKey = 'Load Pattern Definitions - Auto Seismic - User Coefficient'
fieldKeysIncluded = ['Name',
  'IsAuto',
  'XDir',
  'XDirPlusE',
  'XDirMinusE',
  'YDir',
  'YDirPlusE',
  'YDirMinusE',
  'EccRatio',
  'TopStory',
  'BotStory',
  'OverStory',
  'OverDiaph',
  'OverEcc',
  'C',
  'K']

TableData = ['EQ',
  'No',
  'NO',
  'NO',
  'NO',
  'NO',
  'NO',
  'NO',
  '0.05',
  'Story4',
  'Base',
  '',
  '',
  '',
  '0.1',
  '1']

ret = SapModel.DatabaseTables.SetTableForEditingArray(tTableKey, TableVersion,
fieldKeysIncluded, NumberRecords, TableData)
ret=SapModel.DatabaseTables.ApplyEditedTables(True)

ret

[0,
 1,
 0,
 0,
 '********** DATABASE IMPORT DEFINITION DATA **********\r\n\r\nETABS Version 20.3.0 Interactive Database Import\r\nImport Type and Options:  Interactive Database Editing\r\n     Elements Deleted from DB:  Delete object from model\r\n     Other Items Deleted from DB:  Delete item from model\r\n     Elements with Same Name in Model and DB:  Replace object in model\r\n     Other Items with Same Name in Model and DB:  Replace item in model\r\n     Elements (Except Links) in the Same Location:  Allow duplicates in the same location\r\n     Link Elements in the Same Location:  Allow duplicates in the same location\r\n     Items With Same Name in a DB Table:  Overwrite previous item\r\n     Import Control for Errors:  Abort import after 20 errors\r\n     Import Control for Warnings:  Abort import after 200 warnings\r\n\r\nImport started at \r\n\r\nImport completed at 01:12:52\r\nNumber of errors:  1\r\nNumber of warnings:  0\r\nNumber of informational messages:  0\r\n\r\n\r\n*